# EDA and data cleaning

In this notebook, we will go over the data we have and perform EDA and data cleaning.

In [1]:
# imports
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
import re


## Dropping unnecessary columns
We will start our cleaning process by dropping the columns we are sure we dont need.

In [2]:
# read the preprocessed data
df = pd.read_csv('./../data/austin_listings_processed.csv')
print(f'the size of our data is {df.shape}')
df.head(2)

the size of our data is (47037, 81)


C:\Users\masou\AppData\Local\Temp\ipykernel_9492\3186278372.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./../data/austin_listings_processed.csv')


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,occ_rate_calendar,active_duration_days,occ_rate_70,occ_rate_50,occ_rate_30,time_quarter
0,5456,https://www.airbnb.com/rooms/5456,20231215200307,2023-12-16,city scrape,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,NaN,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,1,0,0,3.71,0.3,5390.0,0.700000,0.700000,0.7,Q4
1,5769,https://www.airbnb.com/rooms/5769,20231215200307,2023-12-16,previous scrape,Home in Austin · ★4.91 · 1 bedroom · 1 bed · 1...,NaN,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,...,0,1,0,1.76,0.7,5404.0,0.388601,0.544041,0.7,Q4


In [ ]:
# print list of the columns
print(list(df.columns))


Let us retain the columns we might wanna use later. Only drop the columns that won't be used for sure. 

In [ ]:

columns_to_keep = ['id', 'source', 'name', 'description','neighborhood_overview',
                   'host_is_superhost', 'neighbourhood_cleansed', 'latitude',
                   'longitude', 'property_type', 'room_type', 'accommodates',
                   'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
                   'minimum_nights', 'maximum_nights', 'number_of_reviews',
                   'review_scores_rating','occ_rate_50', 'time_quarter',
                   ]
df = df[columns_to_keep]
# rename columns if needed
df.rename(columns={'neighbourhood_cleansed': 'zipcode',
                   'occ_rate_50': 'occupancy_rate'}, inplace=True)
df.head(3)


Let us start our analysis by looking at the nans.

In [ ]:
df.isna().sum().sort_values(ascending=False)

Since price is a very import feature in our data, let us dive deeper into why it has 2000 missing values.

## Missing prices

In [ ]:
df[df['price'].isna()].head(5)

Let us see if we can find further insight towards what listings have missing prices.

In [ ]:
df[df['price'].isna()].describe(include='object')

In [ ]:
df[df['price'].isna()].describe()


In [ ]:
df[df['price'].isna()]['id'].nunique()

As we can see, all of the missing prices have happened during data scraping in Q4 so there could be some issues at that time (they all have unique id's as well). Looking further, we can see that none of the listings with missing price have description either but this may not be a significant finding because description column has a lot of missing values, in addition to the ones with no price (similar observation for bedrooms).   

Let us see what is the total number of unique id's for our listings.

In [ ]:
df['id'].nunique()

This means we will have a lot of listings that have been listed at different times. We might be able to use price information for that specific property by looking at the prices it was listed for at other times. 

In [ ]:
missing_price_ids = list(df[df['price'].isna()]['id'].unique())

In [ ]:
df[df['id'] == missing_price_ids[0]]

Let us replace the missing price values with that of the same listing on other times periods. 

In [ ]:
#convert currency columns to float when avaialble
df['price'] = df['price'].str.replace('[\$,]', '', regex=True).astype(float)
df['price'].dtype

In [ ]:
# make a dictionary where the key is the index id for the missing prices and value is the average price for that listing
# based on other times of the year
avrg_price = {}
for i in missing_price_ids:
    avrg_price[i] = df[df['id'] == i]['price'].mean()

# replace the missings with average from other dates
df.loc[df['price'].isna(),'price'] = df[df['price'].isna()].apply(lambda x: avrg_price[x['id']], axis=1)

In [ ]:
df['price'].isna().sum()

As we can see, we have replaced most of the listings with missing prices with some values from the same listing at other times (data missing not at random). Now, we can go ahead and drop the remaining nan's in the price column. 

In [ ]:
df = df.dropna(subset=['price'])
df.isna().sum().sort_values(ascending=False)

## Missing values in bedrooms

An interesting trend we are observing is that we already know all airbnb listings should have information about the number of bedrooms. However, we have a lot of listings without any data for the bedrooms. Similar to what we did with price, let us see if we can find information about the number of bedrooms from earlier listings of the same property and use that in our data.


In [ ]:
missing_bedroom_id = df[df['bedrooms'].isna()]['id'].unique()
missing_bedroom = {}
# go over listings with missibg bedroom info and see if the same listing has some bedrooms reported earlier or later in time
for i in missing_bedroom_id:
    # if there are different number of bedrooms reported for the same property, pick up the smaller one (arbitrary decision)
    min_bd = df[df['id'] == i ]['bedrooms'].min()
    missing_bedroom[i] = min_bd

# now replace the nan's in missing bedroom in the same listing has bedrooms reported somewhere else
df.loc[df['bedrooms'].isna(),'bedrooms'] = df[df['bedrooms'].isna()].apply(lambda x: missing_bedroom[x['id']], axis=1)
     

In [ ]:
df['bedrooms'].isna().sum()

This reduces the number of missing bedrooms to 4431.

When we look at the listing name column, we can see very interesting information about the home, number of bedrooms, beds, and bathrooms are provided. 

In [ ]:
df.iloc[0]['name']

From this pattern, we can see that the '.' character can be used to separate the string upon. Let us first make sure that all the names in our listing follow the same convention. 

In [ ]:
df['name'].apply(lambda x: len(x.split('·'))).value_counts()

As we can see, there are a lot of listings that don't follow this naming convenction, let's take a look at them.

In [ ]:
df[df['name'].apply(lambda x: len(x.split('·'))) == 1].head(2)

Let us look at the first listing from the list above.

In [ ]:
df[df['id'] ==5456]

As we can see, this property has some other listings as well and it looks like after Q1, airbnb has decided to follow the listing convention of using '.' in their names. let us check another listing. 

In [ ]:
df[df['id'] ==5769]

Let us see how many missing values for bedrooms do we have for listing where the name is using the new convention of using '.' and indicating the number of beds and baths. 

In [ ]:
print('number of listing with missing bedrooms that use "." to separate the number of rooms in the listing name (5 dots)')
print(df[df['name'].apply(lambda x: len(x.split('·'))) == 5]['bedrooms'].isna().sum())

In [ ]:
print('number of listing with missing bedrooms that use "." to separate the number of rooms in the listing name (4 dots)')
print(df[df['name'].apply(lambda x: len(x.split('·'))) == 4]['bedrooms'].isna().sum())

In [ ]:
print('number of listing with missing bedrooms that use "." to separate the number of rooms in the listing name (3 dots)')
print(df[df['name'].apply(lambda x: len(x.split('·'))) == 3]['bedrooms'].isna().sum())

In [ ]:
print('number of listing with missing bedrooms that use "." to separate the number of rooms in the listing name (2 dots)')
print(df[df['name'].apply(lambda x: len(x.split('·'))) == 2]['bedrooms'].isna().sum())

In [ ]:
print('number of listing with missing bedrooms that use "." to separate the number of rooms in the listing name (1 dots)')
print(df[df['name'].apply(lambda x: len(x.split('·'))) == 1]['bedrooms'].isna().sum())

It looks like most of the listings that have the number of bedrooms missing should have some indication of the number of bedrooms in their name listing. We will have to regex to extract the bedroom information from this data. The following regex will extract the first number before the words 'bedroom', 'Bedroom', 'bedrooms', or 'Bedrooms'. 

In [ ]:
df['bedrooms_extracted'] = df['name'].apply(lambda x: re.search(r'\D*(\d+\.\d+|\d+)\D*(?:bedroom|bedrooms|Bedroom|Bedrooms)',x).group(1) if 
                                            re.search(r'\D*(\d+\.\d+|\d+)\D*(?:bedroom|bedrooms|Bedroom|Bedrooms)',x) else np.nan)                           


In [ ]:
# convert the string into int if they are not nan
df['bedrooms_extracted'] = pd.to_numeric(df['bedrooms_extracted'], errors='coerce').astype('Int64')


In [ ]:
df['bedrooms_extracted'].value_counts()

In [ ]:
df['bedrooms_extracted'].isna().sum()

Now, let us take a look at the column beedrooms and bedrooms_extracted together and see how many listings dont have values in either of these columns.

In [ ]:
df[df['bedrooms'].isna() & df['bedrooms_extracted'].isna()].shape

In [ ]:
df[df['bedrooms'].isna() & df['bedrooms_extracted'].isna()].head(5)

It seems like a lot of these places could be studios. What we can do here now is to see what listing has the word studio in it and then assign a value of 0 to its bedrooms_extracted column value. 

In [ ]:
df.loc[(df['bedrooms_extracted'].isna() &
       df['name'].str.contains('Studio|studio|STUDIO')),'bedrooms_extracted'] = 0
df[df['bedrooms_extracted'].isna() &
   df['name'].str.contains('Studio|studio|STUDIO')
   ].shape

Again, going back to the listings with no beedrooms reported in the bedrooms or bedrooms_extracted columns.

In [ ]:
df[df['bedrooms'].isna() & df['bedrooms_extracted'].isna()].shape

If we exclude the listing that are private or shared rooms (we will not be modeling those) we have:

In [ ]:
df[df['bedrooms_extracted'].isna() & 
   df['bedrooms'].isna() & 
   df['room_type'].str.contains('Entire home/apt')].shape

We will only have 455 important listing with no bedroom information (we will have to drop these listings). At this time, let us combine the data from bedrooms and bedroom_extracted columns. To do that, we will first check if we have any information about bedroom in the bedroom column itself, if not, we will use that of bedroom extracted. 

In [ ]:
df['bedrooms'] = df.apply(lambda x: x['bedrooms'] if not np.isnan(x['bedrooms']) else x['bedrooms_extracted'], axis=1)
df.drop(columns=['bedrooms_extracted'], inplace=True)
df['bedrooms'].isna().sum()

Similar to what we did before, let us make a dictionary with listings that have 4 or 5 sections in their name and see if we can replace the name of the ones with less sections in their name with this new convention. 

In [ ]:
# make a dictionary where the key is the index id for the missing prices and value is the average price for that listing
# based on other times of the year
name_4_sections_id = df[df['name'].apply(lambda x: len(x.split('·'))) == 4]['id'].unique()
name_4_sections = {}


In [ ]:

for i in missing_price_ids:
    avrg_price[i] = df[df['id'] == i]['price'].mean()

# replace the missings with average from other dates
df.loc[df['price'].isna(),'price'] = df[df['price'].isna()].apply(lambda x: avrg_price[x['id']], axis=1)

In [ ]:
df['name'].apply(lambda x: len(x.split('·')))

In [ ]:
df[df['name'].apply(lambda x: len(x.split('·'))) == 3].head(3)

In [ ]:
df[df['id'] ==978089]

In [ ]:
re.findall(r'\d+', df.iloc[0]['name'].split('·')[2])

In [ ]:
re.findall(r'\b\d+\.\d+\b', df.iloc[0]['name'].split('·')[1])

In [ ]:
re.findall(r'(\d+\.\d+|\d+)\s*(?:year|Years)', '2.5 2.5 Years')[0]

In [ ]:
re.findall(r'\D*(\d+\.\d+|\d+)\D*(?:year|Year)', '2.55 aa-- Years')

In [ ]:
re.match(r'\D*(\d+\.\d+|\d+)\D*(?:year|Year)', '2.55 aa-- Years')